In [1]:
# Import numpy and pandas
# Import categorical encoding, necessary layers and Model from keras

In [1]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model

In [ ]:
# Load datasets into pandas

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# I set some parameters:
# m  = number of training examples,
# dt = total number of pixels,
# d  = number of pixels on dimension of the matrix (square root of d2),
# n  = number of classes

In [4]:
m=train.shape[0]
dt=train.shape[1]-1
d=int(np.sqrt(dt))
n=len(train.label.value_counts())
print([m,dt,d,n])

[42000, 784, 28, 10]


In [12]:
# I check that neither the training nor the test sets have missing values

In [12]:
test.columns[pd.concat([train.drop('label',axis=1),test]).isnull().sum()>0]

Index([], dtype='object')

In [14]:
# I check that there is an even distribution of training examples across the different classes

In [51]:
train['label'].value_counts(sort=False)

0    4132
1    4684
2    4177
3    4351
4    4072
5    3795
6    4137
7    4401
8    4063
9    4188
Name: label, dtype: int64

In [16]:
# Some statistics on the pixel values. 
# I compute Pixel_stats for each pixel, using all training example. 
# Then I get statistics for each Pixel_stat.
# For example, the minimum value for each pixel is always 0, but the maximum value is not always 255.

In [17]:
train.drop('label',axis=1).describe().T.drop('count',axis=1).describe().drop('count',axis=0).add_prefix('Pixel_')

,Pixel_mean,Pixel_std,Pixel_min,Pixel_25%,Pixel_50%,Pixel_75%,Pixel_max
mean,33.408911,49.307334,0.0,0.0,12.241709,62.629145,217.676020
std,42.573157,44.174709,0.0,0.0,33.240031,99.211592,83.830621
min,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
25%,0.129470,4.626490,0.0,0.0,0.000000,0.000000,255.000000
50%,7.231500,36.771597,0.0,0.0,0.000000,0.000000,255.000000
75%,69.034262,100.692231,0.0,0.0,0.000000,153.187500,255.000000
max,139.826143,113.850143,0.0,0.0,173.000000,253.000000,255.000000


In [22]:
# I get the dataset ready for usage with keras.
# Max: maximum pixel value in the training dataset
# x: array of pixel values from training dataset, normalized by Max
# y: labels transformed into n-dimensional array
# x_test: array of pixel values from test dataset, normalized by Max

In [52]:
Max=train.max().max()
x=train.iloc[:,1:].values/Max
y=to_categorical(train.label,n)
x_test=test.values/Max
print([x.shape,x.max(),y.shape])
print([x_test.shape,x_test.max()])

[(42000, 784), 1.0, (42000, 10)]
[(28000, 784), 1.0]


In [53]:
# I check that the categorical encoding of y with '1' at position 'i' of the n-dim array corresponds to label 'i'

In [54]:
[set(train.label[np.argmax(y,axis=1)==i])=={i} for i in range(10)]

[True, True, True, True, True, True, True, True, True, True]

In [56]:
# I define a fully connected network.
# Each hidden layer uses ReLU as activation function, and I use dropout after each of them for regularization
# The number of neurons in each layer is given by the input array dims
# The output layer has n neurons and uses softmax activation

In [55]:
def DenseModel(dims,dropout_rate):
    X_input=Input(shape=(dt))
    X=X_input
    for dim in dims:
        X=Dense(dim,activation='relu')(X)
        X=Dropout(dropout_rate)(X)
    X_output=Dense(n,activation='softmax')(X)
    model=Model(inputs=X_input,outputs=X_output)
    return model

In [ ]:
# The first model I create has 5 hidden layers with dimensions (392, 196, 196, 98, 98), and 35% dropout rate.
# I fit the model with 20 epochs of training, batch size of 128 examples, and 20% of the dataset left for validation

In [87]:
dims=[dt//2,dt//4,dt//4,dt//8,dt//8]
model1=DenseModel(dims,0.35)
model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model1.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_20 (Dense)             (None, 392)               307720    
_________________________________________________________________
dropout_17 (Dropout)         (None, 392)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 196)               77028     
_________________________________________________________________
dropout_18 (Dropout)         (None, 196)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 196)               38612     
_________________________________________________________________
dropout_19 (Dropout)         (None, 196)              

In [88]:
model1.fit(x,y,batch_size=128,epochs=20,validation_split=0.2)

Epoch 1/20
263/263 [==============================] - 2s 9ms/step - loss: 0.8913 - accuracy: 0.6996 - val_loss: 0.2332 - val_accuracy: 0.9336
Epoch 2/20
263/263 [==============================] - 2s 9ms/step - loss: 0.3041 - accuracy: 0.9207 - val_loss: 0.1626 - val_accuracy: 0.9564
Epoch 3/20
263/263 [==============================] - 2s 9ms/step - loss: 0.2292 - accuracy: 0.9410 - val_loss: 0.1459 - val_accuracy: 0.9606
Epoch 4/20
263/263 [==============================] - 2s 8ms/step - loss: 0.1865 - accuracy: 0.9516 - val_loss: 0.1289 - val_accuracy: 0.9652
Epoch 5/20
263/263 [==============================] - 2s 9ms/step - loss: 0.1655 - accuracy: 0.9581 - val_loss: 0.1237 - val_accuracy: 0.9670
Epoch 6/20
263/263 [==============================] - 3s 10ms/step - loss: 0.1443 - accuracy: 0.9628 - val_loss: 0.1250 - val_accuracy: 0.9701
Epoch 7/20
263/263 [==============================] - 3s 11ms/step - loss: 0.1260 - accuracy: 0.9676 - val_loss: 0.1347 - val_accuracy: 0.9668
Epoc

In [ ]:
# I set the prediction to the the class with the highest probability in the output layer (softmax) 

In [89]:
pred1=pd.DataFrame({'ImageId':range(1,len(x_test)+1),'Label':model1.predict(x_test).argmax(axis=1)}).set_index('ImageId')
pred1.to_csv('pred1.csv')

In [ ]:
# The second model has 7 hidden layers with dimensions (784, 784, 392, 392, 196, 196, 98), and 35% dropout rate.
# I fit the model with 25 epochs of training, batch size of 128 examples, and 20% of the dataset left for validation

In [85]:
dims=[dt,dt,dt//2,dt//2,dt//4,dt//4,dt//8]
model2=DenseModel(dims,0.35)
model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model2.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_12 (Dense)             (None, 784)               615440    
_________________________________________________________________
dropout_10 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 784)               615440    
_________________________________________________________________
dropout_11 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 392)               307720    
_________________________________________________________________
dropout_12 (Dropout)         (None, 392)              

In [86]:
model2.fit(x,y,batch_size=128,epochs=25,validation_split=0.2)

Epoch 1/25
263/263 [==============================] - 8s 29ms/step - loss: 0.9188 - accuracy: 0.6856 - val_loss: 0.2254 - val_accuracy: 0.9426
Epoch 2/25
263/263 [==============================] - 8s 29ms/step - loss: 0.2857 - accuracy: 0.9308 - val_loss: 0.1631 - val_accuracy: 0.9606
Epoch 3/25
263/263 [==============================] - 8s 30ms/step - loss: 0.2111 - accuracy: 0.9504 - val_loss: 0.1388 - val_accuracy: 0.9665
Epoch 4/25
263/263 [==============================] - 8s 30ms/step - loss: 0.1796 - accuracy: 0.9579 - val_loss: 0.1379 - val_accuracy: 0.9665
Epoch 5/25
263/263 [==============================] - 7s 28ms/step - loss: 0.1563 - accuracy: 0.9629 - val_loss: 0.1214 - val_accuracy: 0.9698
Epoch 6/25
263/263 [==============================] - 8s 31ms/step - loss: 0.1310 - accuracy: 0.9687 - val_loss: 0.1386 - val_accuracy: 0.9686
Epoch 7/25
263/263 [==============================] - 9s 34ms/step - loss: 0.1261 - accuracy: 0.9716 - val_loss: 0.1134 - val_accuracy: 0.9745

In [93]:
pred2=pd.DataFrame({'ImageId':range(1,len(x_test)+1),'Label':model2.predict(x_test).argmax(axis=1)}).set_index('ImageId')
pred2.to_csv('pred2.csv')

In [ ]:
# I check that almost 2% of the predictions differ compared with the previous model

In [100]:
pred2.join(pred1, lsuffix='1',rsuffix='2').apply(lambda x: x.Label1==x.Label2,axis=1).value_counts()/len(x_test)

True     0.980321
False    0.019679
dtype: float64

In [ ]:
# Third model has 9 hidden layers, dimensions are (784, 784, 784, 392, 392, 392, 196, 196, 196), and 35% dropout rate.
# I fit the model with 30 epochs of training, batch size of 128 examples, and 20% of the dataset left for validation

In [102]:
dims=[dt,dt,dt,dt//2,dt//2,dt//2,dt//4,dt//4,dt//4]
model3=DenseModel(dims,0.35)
model3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model3.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_26 (Dense)             (None, 784)               615440    
_________________________________________________________________
dropout_22 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 784)               615440    
_________________________________________________________________
dropout_23 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 784)               615440    
_________________________________________________________________
dropout_24 (Dropout)         (None, 784)              

In [103]:
model3.fit(x,y,batch_size=128,epochs=30,validation_split=0.2)

Epoch 1/30
263/263 [==============================] - 15s 56ms/step - loss: 1.3537 - accuracy: 0.4872 - val_loss: 0.3547 - val_accuracy: 0.8770
Epoch 2/30
263/263 [==============================] - 12s 46ms/step - loss: 0.3706 - accuracy: 0.9076 - val_loss: 0.2177 - val_accuracy: 0.9486
Epoch 3/30
263/263 [==============================] - 12s 46ms/step - loss: 0.2583 - accuracy: 0.9397 - val_loss: 0.1725 - val_accuracy: 0.9615
Epoch 4/30
263/263 [==============================] - 13s 49ms/step - loss: 0.2067 - accuracy: 0.9519 - val_loss: 0.1862 - val_accuracy: 0.9573
Epoch 5/30
263/263 [==============================] - 13s 49ms/step - loss: 0.1882 - accuracy: 0.9580 - val_loss: 0.1368 - val_accuracy: 0.9679
Epoch 6/30
263/263 [==============================] - 13s 49ms/step - loss: 0.1627 - accuracy: 0.9623 - val_loss: 0.1540 - val_accuracy: 0.9673
Epoch 7/30
263/263 [==============================] - 14s 51ms/step - loss: 0.1543 - accuracy: 0.9651 - val_loss: 0.1356 - val_accuracy:

In [105]:
pred3=pd.DataFrame({'ImageId':range(1,len(x_test)+1),'Label':model3.predict(x_test).argmax(axis=1)}).set_index('ImageId')
pred3.to_csv('pred3.csv')

In [ ]:
# I check that almost 2.5% of the predictions differ compared with both model 1 and 2.

In [106]:
pred3.join(pred1, lsuffix='1',rsuffix='2').apply(lambda x: x.Label1==x.Label2,axis=1).value_counts()/len(x_test)

True     0.975929
False    0.024071
dtype: float64

In [107]:
pred3.join(pred2, lsuffix='1',rsuffix='2').apply(lambda x: x.Label1==x.Label2,axis=1).value_counts()/len(x_test)

True     0.975036
False    0.024964
dtype: float64

In [ ]:
# Now I create the datasets for convolutional neural networks
# x_conv: 28 by 28 array of pixel values from training dataset, normalized by Max
# x_conv_test: 28 by 28 array of pixel values from test dataset, normalized by Max

In [112]:
x_conv=train.values[:,1:].reshape((m,d,d,1))/Max
x_conv_test=test.values.reshape((-1,d,d,1))/Max
print([x_conv.shape,x_conv.max()])
print([x_conv_test.shape,x_conv_test.max()])

[(42000, 28, 28, 1), 1.0]
[(28000, 28, 28, 1), 1.0]


In [113]:
# I define a convolutional neural network.
# Each hidden layer uses ReLU as activation function, and I use dropout after each of them for regularization
# The number of filters in each convolutional layer, the kernel sizes and strides are given as input arrays
# I use square filters, with the same stride on both dimensions. I always use 'same' padding
# After convolutional layers I flatten and apply a fully connected layer, with dimension given as input
# The output layer has n neurons and uses softmax activation

In [114]:
def ConvModel(dims,kernels,strides,dim_dense,dropout_rate):
    data=np.array([dims,kernels,strides]).T
    X_input=Input(shape=(d,d,1))
    X=X_input
    for dim,kernel,stride in data:
        X=Conv2D(dim,kernel_size=(kernel,kernel),strides=(stride,stride),padding='same',activation='relu')(X)
        X=Dropout(dropout_rate)(X)
    X=Flatten()(X)
    X=Dense(dim_dense,activation='relu')(X)
    X_output=Dense(n,activation='softmax')(X)
    model=Model(inputs=X_input,outputs=X_output)
    return model

In [ ]:
# Generically I increase the number of filters as I reduce the size of the matrices. All kernels are 3x3.
# I start with 28x28x1 and go to 14x14x32 with a stride of 2.
# I then use 'same' padding and unit stride to stay with 14x14 window, but increase to 64 filters
# Finally I use stride of 2 again to go to 7x7x128.
# I use 128 neurons in the hidden dense layer.
# I fit the model with 15 epochs of training, 35% dropout, and 20% of the dataset left for validation

In [121]:
convmodel1=ConvModel([32,64,128],[3,3,3],[2,1,2],128,0.35)
convmodel1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
convmodel1.summary()

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
dropout_43 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
dropout_44 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
dropout_45 (Dropout)         (None, 7, 7, 128)       

In [122]:
convmodel1.fit(x_conv,y_conv,batch_size=128,epochs=15,validation_split=0.2)

Epoch 1/15
263/263 [==============================] - 35s 133ms/step - loss: 0.3503 - accuracy: 0.8931 - val_loss: 0.0960 - val_accuracy: 0.9677
Epoch 2/15
263/263 [==============================] - 35s 133ms/step - loss: 0.1030 - accuracy: 0.9688 - val_loss: 0.0594 - val_accuracy: 0.9804
Epoch 3/15
263/263 [==============================] - 40s 153ms/step - loss: 0.0759 - accuracy: 0.9761 - val_loss: 0.0485 - val_accuracy: 0.9850
Epoch 4/15
263/263 [==============================] - 34s 129ms/step - loss: 0.0557 - accuracy: 0.9826 - val_loss: 0.0465 - val_accuracy: 0.9851
Epoch 5/15
263/263 [==============================] - 32s 122ms/step - loss: 0.0495 - accuracy: 0.9840 - val_loss: 0.0389 - val_accuracy: 0.9874
Epoch 6/15
263/263 [==============================] - 32s 124ms/step - loss: 0.0427 - accuracy: 0.9857 - val_loss: 0.0445 - val_accuracy: 0.9856
Epoch 7/15
263/263 [==============================] - 31s 118ms/step - loss: 0.0359 - accuracy: 0.9888 - val_loss: 0.0395 - val_ac

In [123]:
convpred1=pd.DataFrame({'ImageId':range(1,len(x_conv_test)+1),'Label':convmodel1.predict(x_conv_test).argmax(axis=1)}).set_index('ImageId')
convpred1.to_csv('convpred1.csv')

In [ ]:
# I use more hidden layers, but still all kernels are 3x3.
# Number of filters is 32,64 and 128, twice each. 
# I half the size of the window every two layers (to 14x14, 7x7 and 4x4)
# I use 256 neurons in the hidden dense layer.
# I fit the model with 20 epochs of training, 35% dropout, and 20% of the dataset left for validation

In [125]:
convmodel2=ConvModel([32,32,64,64,128,128],[3,3,3,3,3,3],[2,1,2,1,2,1],256,0.35)
convmodel2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
convmodel2.summary()

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
dropout_52 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 14, 14, 32)        9248      
_________________________________________________________________
dropout_53 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
dropout_54 (Dropout)         (None, 7, 7, 64)        

In [126]:
convmodel2.fit(x_conv,y_conv,batch_size=128,epochs=20,validation_split=0.2)

Epoch 1/20
263/263 [==============================] - 44s 168ms/step - loss: 0.5554 - accuracy: 0.8110 - val_loss: 0.1132 - val_accuracy: 0.9643
Epoch 2/20
263/263 [==============================] - 46s 175ms/step - loss: 0.1605 - accuracy: 0.9493 - val_loss: 0.0755 - val_accuracy: 0.9750
Epoch 3/20
263/263 [==============================] - 46s 174ms/step - loss: 0.1201 - accuracy: 0.9609 - val_loss: 0.0618 - val_accuracy: 0.9786
Epoch 4/20
263/263 [==============================] - 43s 163ms/step - loss: 0.0985 - accuracy: 0.9689 - val_loss: 0.0492 - val_accuracy: 0.9836
Epoch 5/20
263/263 [==============================] - 43s 165ms/step - loss: 0.0863 - accuracy: 0.9739 - val_loss: 0.0489 - val_accuracy: 0.9821
Epoch 6/20
263/263 [==============================] - 44s 168ms/step - loss: 0.0793 - accuracy: 0.9757 - val_loss: 0.0461 - val_accuracy: 0.9843
Epoch 7/20
263/263 [==============================] - 42s 160ms/step - loss: 0.0705 - accuracy: 0.9779 - val_loss: 0.0446 - val_ac

In [127]:
convpred2=pd.DataFrame({'ImageId':range(1,len(x_conv_test)+1),'Label':convmodel2.predict(x_conv_test).argmax(axis=1)}).set_index('ImageId')
convpred2.to_csv('convpred2.csv')

In [ ]:
# I try a simpler network now, with just two concolutional layers, dimensions are 32 and 64, windows 14x14 and 7x7.
# Dense layer with 128 neurons. I use 13 epochs of training, 35% dropout rate, and 20% train/dev partitioning

In [128]:
convmodel3=ConvModel([32,64],[3,3],[2,2],128,0.35)
convmodel3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
convmodel3.summary()

Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
dropout_58 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
dropout_59 (Dropout)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_50 (Dense)             (None, 128)             

In [129]:
convmodel3.fit(x_conv,y_conv,batch_size=128,epochs=13,validation_split=0.2)

Epoch 1/13
263/263 [==============================] - 11s 41ms/step - loss: 0.4417 - accuracy: 0.8694 - val_loss: 0.1500 - val_accuracy: 0.9555
Epoch 2/13
263/263 [==============================] - 9s 34ms/step - loss: 0.1568 - accuracy: 0.9527 - val_loss: 0.0940 - val_accuracy: 0.9710
Epoch 3/13
263/263 [==============================] - 10s 40ms/step - loss: 0.1080 - accuracy: 0.9662 - val_loss: 0.0680 - val_accuracy: 0.9773
Epoch 4/13
263/263 [==============================] - 10s 39ms/step - loss: 0.0832 - accuracy: 0.9728 - val_loss: 0.0570 - val_accuracy: 0.9813
Epoch 5/13
263/263 [==============================] - 11s 41ms/step - loss: 0.0690 - accuracy: 0.9785 - val_loss: 0.0608 - val_accuracy: 0.9799
Epoch 6/13
263/263 [==============================] - 10s 36ms/step - loss: 0.0601 - accuracy: 0.9807 - val_loss: 0.0460 - val_accuracy: 0.9852
Epoch 7/13
263/263 [==============================] - 9s 34ms/step - loss: 0.0520 - accuracy: 0.9837 - val_loss: 0.0492 - val_accuracy: 0

In [130]:
convpred3=pd.DataFrame({'ImageId':range(1,len(x_conv_test)+1),'Label':convmodel3.predict(x_conv_test).argmax(axis=1)}).set_index('ImageId')
convpred3.to_csv('convpred3.csv')

In [ ]:
# I define a new convolutional neural network, with batch normalization layers between convolution and activation

In [131]:
def ConvModel2(dims,kernels,strides,dim_dense,dropout_rate):
    data=np.array([dims,kernels,strides]).T
    X_input=Input(shape=(d,d,1))
    X=X_input
    for dim,kernel,stride in data:
        X=Conv2D(dim,kernel_size=(kernel,kernel),strides=(stride,stride),padding='same',use_bias=False)(X)
        X=BatchNormalization()(X)
        X=Activation('relu')(X)
        X=Dropout(dropout_rate)(X)
    X=Flatten()(X)
    X=Dense(dim_dense,use_bias=False)(X)
    X=BatchNormalization()(X)
    X=Activation('relu')(X)
    X_output=Dense(n,activation='softmax')(X)
    model=Model(inputs=X_input,outputs=X_output)
    return model

In [ ]:
# Like Convolutional model 1 (4 hidden layers), but with batch normalization

In [133]:
convmodel4=ConvModel2([32,64,128],[3,3,3],[2,1,2],128,0.35)
convmodel4.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
convmodel4.summary()

Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 14, 14, 32)        288       
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
activation_4 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_63 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 14, 14, 64)        18432     
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 64)      

In [134]:
convmodel4.fit(x_conv,y_conv,batch_size=128,epochs=15,validation_split=0.2)

Epoch 1/15
263/263 [==============================] - 45s 171ms/step - loss: 0.2973 - accuracy: 0.9125 - val_loss: 1.1990 - val_accuracy: 0.6181
Epoch 2/15
263/263 [==============================] - 46s 177ms/step - loss: 0.0945 - accuracy: 0.9722 - val_loss: 0.0983 - val_accuracy: 0.9700
Epoch 3/15
263/263 [==============================] - 45s 172ms/step - loss: 0.0665 - accuracy: 0.9801 - val_loss: 0.0460 - val_accuracy: 0.9855
Epoch 4/15
263/263 [==============================] - 41s 154ms/step - loss: 0.0544 - accuracy: 0.9835 - val_loss: 0.0388 - val_accuracy: 0.9874
Epoch 5/15
263/263 [==============================] - 40s 152ms/step - loss: 0.0449 - accuracy: 0.9861 - val_loss: 0.0321 - val_accuracy: 0.9895
Epoch 6/15
263/263 [==============================] - 40s 153ms/step - loss: 0.0401 - accuracy: 0.9872 - val_loss: 0.0318 - val_accuracy: 0.9892
Epoch 7/15
263/263 [==============================] - 48s 184ms/step - loss: 0.0345 - accuracy: 0.9896 - val_loss: 0.0340 - val_ac

In [135]:
convpred4=pd.DataFrame({'ImageId':range(1,len(x_conv_test)+1),'Label':convmodel4.predict(x_conv_test).argmax(axis=1)}).set_index('ImageId')
convpred4.to_csv('convpred4.csv')

In [ ]:
# Like Convolutional model 2 (7 hidden layers), but with batch normalization

In [136]:
convmodel5=ConvModel2([32,32,64,64,128,128],[3,3,3,3,3,3],[2,1,2,1,2,1],256,0.35)
convmodel5.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
convmodel5.summary()

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 14, 14, 32)        288       
_________________________________________________________________
batch_normalization_8 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_66 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 14, 14, 32)        9216      
_________________________________________________________________
batch_normalization_9 (Batch (None, 14, 14, 32)      

In [137]:
convmodel5.fit(x_conv,y_conv,batch_size=128,epochs=20,validation_split=0.2)

Epoch 1/20
263/263 [==============================] - 58s 220ms/step - loss: 0.6265 - accuracy: 0.7922 - val_loss: 0.3228 - val_accuracy: 0.9006
Epoch 2/20
263/263 [==============================] - 58s 222ms/step - loss: 0.1764 - accuracy: 0.9438 - val_loss: 0.0847 - val_accuracy: 0.9723
Epoch 3/20
263/263 [==============================] - 52s 199ms/step - loss: 0.1290 - accuracy: 0.9585 - val_loss: 0.0608 - val_accuracy: 0.9780
Epoch 4/20
263/263 [==============================] - 54s 205ms/step - loss: 0.1055 - accuracy: 0.9661 - val_loss: 0.0445 - val_accuracy: 0.9840
Epoch 5/20
263/263 [==============================] - 52s 197ms/step - loss: 0.0911 - accuracy: 0.9708 - val_loss: 0.0428 - val_accuracy: 0.9861
Epoch 6/20
263/263 [==============================] - 54s 205ms/step - loss: 0.0795 - accuracy: 0.9744 - val_loss: 0.0408 - val_accuracy: 0.9869
Epoch 7/20
263/263 [==============================] - 57s 216ms/step - loss: 0.0739 - accuracy: 0.9761 - val_loss: 0.0404 - val_ac

In [140]:
convpred5=pd.DataFrame({'ImageId':range(1,len(x_conv_test)+1),'Label':convmodel5.predict(x_conv_test).argmax(axis=1)}).set_index('ImageId')
convpred5.to_csv('convpred5.csv')

In [ ]:
# Finally, I create yet a third type of model, where there is a pooling layer between activation and dropout.
# The idea is that I will now be able to decrease the window size with pooling instead of using wider strides.

In [138]:
def ConvModel3(dims,kernels,strides,pools,dim_dense,dropout_rate):
    data=np.array([dims,kernels,strides,pools]).T
    X_input=Input(shape=(d,d,1))
    X=X_input
    for dim,kernel,stride,pool in data:
        X=Conv2D(dim,kernel_size=(kernel,kernel),strides=(stride,stride),padding='same',use_bias=False)(X)
        X=BatchNormalization()(X)
        X=Activation('relu')(X)
        X=MaxPooling2D(pool_size=(pool,pool))(X)
        X=Dropout(dropout_rate)(X)
    X=Flatten()(X)
    X=Dense(dim_dense,use_bias=False)(X)
    X=BatchNormalization()(X)
    X=Activation('relu')(X)
    X_output=Dense(n,activation='softmax')(X)
    model=Model(inputs=X_input,outputs=X_output)
    return model

In [ ]:
# Like Convolutional model 1 (4 hidden layers), but with batch normalization and pooling.
# All strides are set to 1, and I pool with 2x2 filters after each convolutional layer.

In [139]:
convmodel6=ConvModel3([32,64,128],[3,3,3],[1,1,1],[2,2,2],128,0.35)
convmodel6.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
convmodel6.summary()

Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 28, 28, 32)        288       
_________________________________________________________________
batch_normalization_15 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
activation_15 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_72 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 14, 14, 64)      

In [141]:
convmodel6.fit(x_conv,y_conv,batch_size=128,epochs=15,validation_split=0.2)

Epoch 1/15
263/263 [==============================] - 55s 208ms/step - loss: 0.4329 - accuracy: 0.8691 - val_loss: 6.2019 - val_accuracy: 0.1131
Epoch 2/15
263/263 [==============================] - 55s 207ms/step - loss: 0.1207 - accuracy: 0.9632 - val_loss: 0.8730 - val_accuracy: 0.7287
Epoch 3/15
263/263 [==============================] - 54s 205ms/step - loss: 0.0864 - accuracy: 0.9730 - val_loss: 0.0535 - val_accuracy: 0.9831
Epoch 4/15
263/263 [==============================] - 60s 229ms/step - loss: 0.0727 - accuracy: 0.9778 - val_loss: 0.0448 - val_accuracy: 0.9854
Epoch 5/15
263/263 [==============================] - 59s 224ms/step - loss: 0.0619 - accuracy: 0.9804 - val_loss: 0.0331 - val_accuracy: 0.9899
Epoch 6/15
263/263 [==============================] - 58s 220ms/step - loss: 0.0551 - accuracy: 0.9828 - val_loss: 0.0318 - val_accuracy: 0.9912
Epoch 7/15
263/263 [==============================] - 59s 225ms/step - loss: 0.0506 - accuracy: 0.9838 - val_loss: 0.0394 - val_ac

In [142]:
convpred6=pd.DataFrame({'ImageId':range(1,len(x_conv_test)+1),'Label':convmodel6.predict(x_conv_test).argmax(axis=1)}).set_index('ImageId')
convpred6.to_csv('convpred6.csv')